In [73]:
import selenium
import time
import pandas as pd

from datetime import datetime
from tqdm import tqdm
from selenium import webdriver as wb

webD=wb.Chrome('chromedriver.exe')
webD.get('https://www.webmotors.com.br/carros/estoque/audi/q5?tipoveiculo=carros&marca1=AUDI&modelo1=Q5&superSealOff=on')

In [74]:
def scroll_to_bottom(driver):

    old_position = 0
    new_position = None

    while new_position != old_position:
        # Get old scroll position
        old_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                 " window.pageYOffset : (document.documentElement ||"
                 " document.body.parentNode || document.body);"))
        # Sleep and Scroll
        time.sleep(1)
        driver.execute_script((
                "var scrollingElement = (document.scrollingElement ||"
                " document.body);scrollingElement.scrollTop ="
                " scrollingElement.scrollHeight;"))
        # Get new position
        new_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                 " window.pageYOffset : (document.documentElement ||"
                 " document.body.parentNode || document.body);"))
    
    #Procura por um botão de carregar mais depois de rolar para baixo
    try:
        #Se encontrar o botão, clica e sinaliza que não estamos no fim da página
        load_more = webD.find_element_by_id('ButtonCarriesMoreCars')
        load_more.click()
        return False
    except:
        #Se não encontrar o botão, estamos no fim da página
        return True

In [75]:
def total_of_cars_page(webD):
    
    #Procura o elemento que mostra o número total de veículos na página
    carsTotalEle = webD.find_element_by_class_name('FoundCars')
    carsTotal = carsTotalEle.text
    
    #Separa o número do resto da string
    start = carsTotal.find("'") + 1
    end = carsTotal.find("carr")
    carsNumber = int(carsTotal[start:end])
    return carsNumber

In [76]:
#Quantos resultados foram encontrados?
NumberOfCars = total_of_cars_page(webD)
print(NumberOfCars)

#Retorna o total de veículos já encontrados
CarsList = webD.find_elements_by_class_name('sc-dNLxif.dcKUo')

while len(CarsList) < NumberOfCars:
    scroll_to_bottom(webD)
    CarsList = webD.find_elements_by_class_name('sc-dNLxif.dcKUo')
    print(len(CarsList))

411
120
240
360
411


In [77]:
Details = []
for car in tqdm(CarsList):
    data = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    link = car.find_element_by_tag_name('a').get_property('href')
    dash = link.rindex("/")
    pos = link.find("?pos=")
    key =  int(link[dash+1:pos])
    carro = car.find_element_by_class_name('sc-kvZOFW.gMydfs').text
    space = carro.find(" ")
    marca = carro[:space]
    modelo = carro[space:]
    versao = car.find_element_by_class_name('sc-hqyNC.hqiJVw').text
    preco = car.find_element_by_class_name('sc-kgAjT.kCXoJF').text
    preco = float(preco[3:].replace('.',''))
    ano = car.find_elements_by_class_name('sc-hmzhuo.jqkOqr')[0].text
    ano = int(ano[:4])
    quilometragem = car.find_elements_by_class_name('sc-hmzhuo.jqkOqr')[1].text
    quilometragem = int(quilometragem[:len(quilometragem)-3])
    cidade = car.find_element_by_class_name('sc-TOsTZ.izLZaP').text
    cidade = cidade[:len(cidade)-5]
    estado = car.find_element_by_class_name('sc-TOsTZ.izLZaP').text
    estado = estado[len(estado)-2:]
    tempdict = {"Data":data,
            "Key":key,
            "Link":link,
           "Marca":marca,
           "Modelo":modelo,
           "Versão":versao,
           "Preço":preco,
           "Ano":ano,
           "Km":quilometragem,
           "Cidade":cidade,
           "Estado":estado}
    Details.append(tempdict)
    
Details[0]

100%|████████████████████████████████████████████████████████████████████████████████| 411/411 [02:17<00:00,  2.99it/s]


{'Data': '24/09/2020 23:00:13',
 'Key': 33237425,
 'Link': 'https://www.webmotors.com.br/comprar/audi/q5/3-0-tfsi-ambition-v6-24v-gasolina-4p-automatico/4-portas/2012-2013/33237425?pos=a33237425g:&np=1',
 'Marca': 'AUDI',
 'Modelo': ' Q5',
 'Versão': '3.0 TFSI AMBITION V6 24V GASOLINA 4P AUTOMÁTICO',
 'Preço': 105900.0,
 'Ano': 2012,
 'Km': 45412,
 'Cidade': 'Fortaleza',
 'Estado': 'CE'}

In [78]:
db = pd.DataFrame(Details)
db

,Data,Key,Link,Marca,Modelo,Versão,Preço,Ano,Km,Cidade,Estado
0,24/09/2020 23:00:13,33237425,https://www.webmotors.com.br/comprar/audi/q5/3...,AUDI,Q5,3.0 TFSI AMBITION V6 24V GASOLINA 4P AUTOMÁTICO,105900.0,2012,45412,Fortaleza,CE
1,24/09/2020 23:00:14,34194385,https://www.webmotors.com.br/comprar/audi/q5/2...,AUDI,Q5,2.0 TFSI ATTRACTION 16V 225CV GASOLINA 4P AUTO...,116900.0,2016,54100,São Paulo,SP
2,24/09/2020 23:00:14,28874699,https://www.webmotors.com.br/comprar/audi/q5/2...,AUDI,Q5,2.0 TFSI AMBIENTE 16V 211CV GASOLINA 4P AUTOMÁ...,53900.0,2009,61900,São Paulo,SP
3,24/09/2020 23:00:14,32009742,https://www.webmotors.com.br/comprar/audi/q5/2...,AUDI,Q5,2.0 TFSI AMBIENTE 16V 225CV GASOLINA 4P AUTOMÁ...,83000.0,2014,74800,Feira de Santana,BA
4,24/09/2020 23:00:14,33826278,https://www.webmotors.com.br/comprar/audi/q5/2...,AUDI,Q5,2.0 TFSI GASOLINA PRESTIGE S TRONIC,220000.0,2019,3200,Santos,SP
...,...,...,...,...,...,...,...,...,...,...,...
406,24/09/2020 23:02:29,32889911,https://www.webmotors.com.br/comprar/audi/q5/2...,AUDI,Q5,2.0 TFSI GASOLINA PRESTIGE S TRONIC,225000.0,2019,24363,Campo Grande,MS
407,24/09/2020 23:02:29,34269744,https://www.webmotors.com.br/comprar/audi/q5/2...,AUDI,Q5,2.0 TFSI AMBIENTE 16V 211CV GASOLINA 4P AUTOMÁ...,99990.0,2011,43000,São Paulo,SP
408,24/09/2020 23:02:30,30740271,https://www.webmotors.com.br/comprar/audi/q5/3...,AUDI,Q5,3.0 TFSI AMBITION V6 24V GASOLINA 4P AUTOMÁTICO,108800.0,2012,59000,Goiânia,GO
409,24/09/2020 23:02:30,34266952,https://www.webmotors.com.br/comprar/audi/q5/2...,AUDI,Q5,2.0 TFSI GASOLINA PRESTIGE PLUS S TRONIC,349900.0,2019,0,São Bernardo do Campo,SP
